In [3]:
import os

In [6]:
%pwd

'c:\\Users\\Shawn\\Desktop\\TextSummarizer'

In [5]:
os.chdir("../../")

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [8]:
from src.constants import *
from src.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath: Path = CONFIG_FILE_PATH,
                 params_filepath: Path = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config

In [11]:
import os
from src.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

In [25]:
class DataTransformation:
    
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)
    

    def convert_examples_to_features(self, example_batch):
        # Tokenize the input text
        input_encodings = self.tokenizer(
            example_batch['dialogue'],
            max_length=1024,
            truncation=True,
        )

        # Tokenize the target/summary text using `text_target`
        target_encodings = self.tokenizer(
            text_target=example_batch['summary'],
            max_length=128,
            truncation=True,
        )

        return {
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)

        split_lengths = [len(dataset_samsum[split])for split in dataset_samsum]

        print(f"Split lengths: {split_lengths}")
        print(f"Features: {dataset_samsum['train'].column_names}")
        print("\nDialogue:")

        print(dataset_samsum["test"][1]["dialogue"])

        print("\nSummary:")

        print(dataset_samsum["test"][1]["summary"])

        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))
        logger.info(f">>>>> Dataset transformed and saved to: {os.path.join(self.config.root_dir, "samsum_dataset")} <<<<<\n\nx====================x")


In [26]:
try:
    config_manager = ConfigurationManager()
    data_transformation_config = config_manager.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2024-12-27 16:19:45,755: INFO: common: Reading C:\Users\Shawn\Desktop\TextSummarizer\app\config\config.yaml file]
[2024-12-27 16:19:45,759: INFO: common: Reading C:\Users\Shawn\Desktop\TextSummarizer\app\params.yaml file]
[2024-12-27 16:19:45,762: INFO: common: Directory created: app/artifacts]
[2024-12-27 16:19:45,764: INFO: common: Directory created: app/artifacts/data_transformation]
Split lengths: [14732, 819, 818]
Features: ['id', 'dialogue', 'summary']

Dialogue:
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)

Summary:
Eric and Rob are going to watch a stand-up on youtube.


Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 29211.18 examples/s]

[2024-12-27 16:20:08,761: INFO: 837505878: Dataset transformed and saved to: app/artifacts/data_transformation\samsum_dataset]
